In [151]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats
import janitor 
import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm

In [3]:
wrds = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [4]:
crsp_m = wrds.raw_sql("""
                      select a.permno, a.permco, a.mthcaldt, 
                      a.issuertype, a.securitytype, a.securitysubtype, a.sharetype, a.usincflg, 
                      a.primaryexch, a.conditionaltype, a.tradingstatusflg,
                      a.mthret, a.mthretx, a.shrout, a.mthprc
                      from crsp.msf_v2 as a
                      where a.mthcaldt <= '12/31/2019'
                      """, date_cols=['mthcaldt']) 
crsp_m.shape

(4575030, 15)

In [5]:
crsp_m = crsp_m.loc[(crsp_m.sharetype=='NS') & \
                    (crsp_m.securitytype=='EQTY') & \
                    (crsp_m.securitysubtype=='COM') & \
                    (crsp_m.usincflg=='Y') & \
                    (crsp_m.issuertype.isin(['ACOR', 'CORP']))]
crsp_m.shape

(3643940, 15)

In [6]:
crsp_m = crsp_m.loc[(crsp_m.primaryexch.isin(['N', 'A', 'Q'])) & \
                   (crsp_m.conditionaltype =='RW') & \
                   (crsp_m.tradingstatusflg =='A')]
crsp_m.shape

(3583238, 15)

In [7]:
crsp_m[['permco','permno']]=crsp_m[['permco','permno']].astype(int)
crsp_m['jdate']=crsp_m['mthcaldt']+MonthEnd(0)

In [8]:
crsp = crsp_m.copy()
crsp['mthret']=crsp['mthret'].fillna(0)           # Monthly Total Return (MthRet)
crsp['mthretx']=crsp['mthretx'].fillna(0)         # Monthly Return Without Dividends (MthRetx)
crsp['me']=crsp['mthprc']*crsp['shrout']          # Monthly Price (MthPrc)
crsp=crsp.drop(['mthprc','shrout'], axis=1)       # Shares Outstanding (ShrOut)
crsp=crsp.sort_values(by=['jdate','permco','me'])

crsp_summe = crsp.groupby(['jdate','permco'])['me'].sum().reset_index()
crsp_maxme = crsp.groupby(['jdate','permco'])['me'].max().reset_index()
crsp1=pd.merge(crsp, crsp_maxme, how='inner', on=['jdate','permco','me'])
crsp1=crsp1.drop(['me'], axis=1)
crsp2=pd.merge(crsp1, crsp_summe, how='inner', on=['jdate','permco'])
crsp2=crsp2.sort_values(by=['permno','jdate']).drop_duplicates()

In [9]:
crsp2['ffdate']=crsp2['jdate']+MonthEnd(-6)
crsp2['ffyear']=crsp2['ffdate'].dt.year
crsp2['ffmonth']=crsp2['ffdate'].dt.month
crsp2['1+retx']=1+crsp2['mthretx']
crsp2=crsp2.sort_values(by=['permno','mthcaldt'])

In [10]:

crsp2['cumretx']=crsp2.groupby(['permno','ffyear'])['1+retx'].cumprod()
crsp2['lcumretx']=crsp2.groupby(['permno'])['cumretx'].shift(1)
crsp2['lme']=crsp2.groupby(['permno'])['me'].shift(1)

crsp2['count']=crsp2.groupby(['permno']).cumcount()
crsp2['lme']=np.where(crsp2['count']==0, crsp2['me']/crsp2['1+retx'], crsp2['lme'])

mebase=crsp2[crsp2['ffmonth']==1][['permno','ffyear', 'lme']].rename(columns={'lme':'mebase'})

crsp3=pd.merge(crsp2, mebase, how='left', on=['permno','ffyear'])
crsp3['wt']=np.where(crsp3['ffmonth']==1, crsp3['lme'], crsp3['mebase']*crsp3['lcumretx'])

In [13]:
comp = wrds.raw_sql("""
                    select gvkey, datadate, at, pstkl, txditc,
                    pstkrv, seq, pstk 
                    from comp.funda
                    where indfmt='INDL' 
                    and datafmt='STD'
                    and popsrc='D'
                    and consol='C'
                    """, date_cols=['datadate'])
comp['year']=comp['datadate'].dt.year

In [14]:
comp['ps']=np.where(comp['pstkrv'].isnull(), comp['pstkl'], comp['pstkrv'])
comp['ps']=np.where(comp['ps'].isnull(),comp['pstk'], comp['ps'])
comp['ps']=np.where(comp['ps'].isnull(),0,comp['ps'])
comp['txditc']=comp['txditc'].fillna(0)

comp['be']=comp['seq']+comp['txditc']-comp['ps']
comp['be']=np.where(comp['be']>0, comp['be'], np.nan)

comp=comp.sort_values(by=['gvkey','datadate'])
comp['count']=comp.groupby(['gvkey']).cumcount()

comp=comp[['gvkey','datadate','year','be','count']]

In [15]:
ccm=wrds.raw_sql("""
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

In [16]:
ccm1=pd.merge(comp[['gvkey','datadate','be']],ccm,how='left',on=['gvkey'])
ccm1['yearend']=ccm1['datadate']+YearEnd(0)
ccm1['jdate']=ccm1['yearend']+MonthEnd(6)

In [17]:
ccm2=ccm1[(ccm1['jdate']>=ccm1['linkdt'])&(ccm1['jdate']<=ccm1['linkenddt'])]
ccm2=ccm2[['permno','jdate','be']]

be = ccm2.copy()
be['jdate_end'] = be['jdate'] + MonthEnd(12)
be = be[['permno','jdate','be','jdate_end']]

In [18]:
crsp99 = crsp3.conditional_join(
    be,
    ('permno', 'permno', '=='),
    ('jdate', 'jdate', '>='),
    ('jdate', 'jdate_end', '<'),
    how = 'left'
    )

crsp99.drop(columns = {('right','permno'),('right','jdate')}, inplace = True)
crsp99.columns = crsp99.columns.droplevel(0)

In [19]:
crsp99 = crsp99[['permno','jdate','me','mthret','mthretx','be','primaryexch','wt']]
crsp99['beme'] = np.where(crsp99['me'] > 0, crsp99['be'] * 1000 / crsp99['me'], np.nan)
crsp99.sort_values(by=['jdate','permno'],inplace=True)

In [20]:
nyse= crsp99[(crsp99['primaryexch']=='N') & (crsp99['beme']>0) & (crsp99['me']>0)]

In [21]:
nyse_sz=nyse.groupby(['jdate'])['me'].median().to_frame().reset_index().rename(columns={'me':'sizemedn'})
nyse_bm=nyse.groupby(['jdate'])['beme'].describe(percentiles=[0.3, 0.7]).reset_index()
nyse_bm=nyse_bm[['jdate','30%','70%']].rename(columns={'30%':'bm30', '70%':'bm70'})

In [22]:
nyse_breaks = pd.merge(nyse_sz, nyse_bm, how='inner', on=['jdate'])
crsp00 = pd.merge(crsp99, nyse_breaks, how='left', on=['jdate'])

In [23]:
def sz_bucket(row):
    if row['me']==np.nan:
        value=''
    elif row['me']<=row['sizemedn']:
        value='S'
    else:
        value='B'
    return value

def bm_bucket(row):
    if 0<=row['beme']<=row['bm30']:
        value = 'G'
    elif row['beme']<=row['bm70']:
        value='M'
    elif row['beme']>row['bm70']:
        value= 'V'
    else:
        value=''
    return value

In [326]:
### assign size portfolio
crsp00['szport']=np.where((crsp00['beme']>0)&(crsp00['me']>0), crsp00.apply(sz_bucket, axis=1), '')
### assign book-to-market portfolio
crsp00['bmport']=np.where((crsp00['beme']>0)&(crsp00['me']>0), crsp00.apply(bm_bucket, axis=1), '')

In [337]:
### create positivebmeme and nonmissport variable
crsp00['posbm']=np.where((crsp00['beme']>0)&(crsp00['me']>0), 1, 0)
crsp00['nonmissport']=np.where((crsp00['bmport']!=''), 1, 0)

crsp00.sort_values(['permno','jdate'],inplace=True)
crsp11=crsp00[(crsp00['wt']>0)& (crsp00['posbm']==1) & (crsp00['nonmissport']==1)]

In [348]:
crsp11['SV'] = np.where((crsp11['szport'] == 'S') & (crsp11['bmport'] == 'V'), 1, 0)
crsp11['BV'] = np.where((crsp11['szport'] == 'B') & (crsp11['bmport'] == 'V'), 1, 0)
crsp11['SG'] = np.where((crsp11['szport'] == 'S') & (crsp11['bmport'] == 'G'), 1, 0)
crsp11['BG'] = np.where((crsp11['szport'] == 'B') & (crsp11['bmport'] == 'G'), 1, 0)

crsp11['MV'] = crsp11['SV'] + crsp11['BV']
crsp11['MG'] = crsp11['SG'] + crsp11['BG']
crsp11.reset_index(drop=True,inplace=True)

In [358]:
crsp22 = crsp11[['jdate','permno','mthret','beme','SV','BV'	,'SG','BG','MV','MG','me']]
crsp22['mthret'] = crsp22['mthret'] * 100

In [359]:
crsp22.sort_values(['permno','jdate'], inplace=True)
crsp22['ret_ahead'] = crsp22.groupby(['permno'])['mthret'].shift(-1)
crsp33 = crsp22[(crsp22['jdate']>='1963-07-31') & (crsp22['jdate']<='2019-06-30')]

In [360]:
crsp33.to_pickle('data99.pkl')

In [325]:
return_whole.to_pickle('return_whole.pkl')

In [317]:
table1_panelC = corr_matrix
print(table1_panelC)

              MV    BV    SV    MG    BG    SG
Cor(R, RM)  0.85  0.83  0.81  0.97  0.97  0.86
MV          1.00  0.97  0.74 -0.73 -0.73 -0.04
BV          0.97  1.00  0.58 -0.72 -0.69 -0.21
SV          0.74  0.58  1.00 -0.49 -0.59  0.45
MG         -0.73 -0.72 -0.49  1.00  0.97  0.22
BG         -0.73 -0.69 -0.59  0.97  1.00  0.02
SG         -0.04 -0.21  0.45  0.22  0.02  1.00
